In [47]:
import pandas as pd
import numpy as np

In [48]:
df = pd.read_html("https://fbref.com/pt/comps/24/2023/cronograma/2023-Serie-A-Resultados-e-Calendarios")[0]
pd.set_option("display.max_columns", None)
print(df)

      Sem  Dia        Data Horário              Em casa   xG Resultado  xG.1  \
0     1.0  sáb  2023-04-15   16:00         América (MG)  1.1       0–3   3.1   
1     1.0  sáb  2023-04-15   16:00            Palmeiras  3.0       2–1   1.0   
2     1.0  sáb  2023-04-15   18:30  Red Bull Bragantino  2.0       2–1   0.7   
3     1.0  sáb  2023-04-15   18:30        Botafogo (RJ)  0.9       2–1   2.0   
4     1.0  sáb  2023-04-15   18:30       Ath Paranaense  0.8       2–0   1.0   
..    ...  ...         ...     ...                  ...  ...       ...   ...   
421  38.0  qua  2023-12-06   21:30           Fluminense  2.4       2–3   2.4   
422  38.0  qua  2023-12-06   21:30            São Paulo  0.5       1–0   0.5   
423  38.0  qua  2023-12-06   21:30             Cruzeiro  1.6       1–1   1.4   
424  38.0  qua  2023-12-06   21:30        Internacional  1.2       3–1   0.4   
425  38.0  qua  2023-12-06   21:30        Vasco da Gama  1.6       2–1   0.8   

               Visitante  Público      

In [49]:
resultados = df[['Sem', 'Data', 'Em casa', 'Resultado', 'Visitante']]
resultados

,Sem,Data,Em casa,Resultado,Visitante
0,1.0,2023-04-15,América (MG),0–3,Fluminense
1,1.0,2023-04-15,Palmeiras,2–1,Cuiabá
2,1.0,2023-04-15,Red Bull Bragantino,2–1,Bahia
3,1.0,2023-04-15,Botafogo (RJ),2–1,São Paulo
4,1.0,2023-04-15,Ath Paranaense,2–0,Goiás
...,...,...,...,...,...
421,38.0,2023-12-06,Fluminense,2–3,Grêmio
422,38.0,2023-12-06,São Paulo,1–0,Flamengo
423,38.0,2023-12-06,Cruzeiro,1–1,Palmeiras
424,38.0,2023-12-06,Internacional,3–1,Botafogo (RJ)


In [50]:
placar = resultados.loc[:,"Resultado"].str.split("–", n = 1, expand = True)
print(placar)
resultados.loc[:,"CasaResul"] = placar.loc[:,0]
resultados.loc[:,"VisiResul"] = placar.loc[:,1]
resultados = resultados.drop(columns=["Resultado"])
resultados

     0  1
0    0  3
1    2  1
2    2  1
3    2  1
4    2  0
..  .. ..
421  2  3
422  1  0
423  1  1
424  3  1
425  2  1

[426 rows x 2 columns]


C:\Users\lccalmon\AppData\Local\Temp\ipykernel_19388\3424367926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados.loc[:,"CasaResul"] = placar.loc[:,0]
C:\Users\lccalmon\AppData\Local\Temp\ipykernel_19388\3424367926.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados.loc[:,"VisiResul"] = placar.loc[:,1]


,Sem,Data,Em casa,Visitante,CasaResul,VisiResul
0,1.0,2023-04-15,América (MG),Fluminense,0,3
1,1.0,2023-04-15,Palmeiras,Cuiabá,2,1
2,1.0,2023-04-15,Red Bull Bragantino,Bahia,2,1
3,1.0,2023-04-15,Botafogo (RJ),São Paulo,2,1
4,1.0,2023-04-15,Ath Paranaense,Goiás,2,0
...,...,...,...,...,...,...
421,38.0,2023-12-06,Fluminense,Grêmio,2,3
422,38.0,2023-12-06,São Paulo,Flamengo,1,0
423,38.0,2023-12-06,Cruzeiro,Palmeiras,1,1
424,38.0,2023-12-06,Internacional,Botafogo (RJ),3,1


In [51]:
resultados["Vencedor"] = np.where(resultados["CasaResul"] > resultados["VisiResul"], "C", 
                                  np.where(resultados["CasaResul"] < resultados["VisiResul"], "V", "E"))
resultados

,Sem,Data,Em casa,Visitante,CasaResul,VisiResul,Vencedor
0,1.0,2023-04-15,América (MG),Fluminense,0,3,V
1,1.0,2023-04-15,Palmeiras,Cuiabá,2,1,C
2,1.0,2023-04-15,Red Bull Bragantino,Bahia,2,1,C
3,1.0,2023-04-15,Botafogo (RJ),São Paulo,2,1,C
4,1.0,2023-04-15,Ath Paranaense,Goiás,2,0,C
...,...,...,...,...,...,...,...
421,38.0,2023-12-06,Fluminense,Grêmio,2,3,V
422,38.0,2023-12-06,São Paulo,Flamengo,1,0,C
423,38.0,2023-12-06,Cruzeiro,Palmeiras,1,1,E
424,38.0,2023-12-06,Internacional,Botafogo (RJ),3,1,C


In [65]:
#verificando rodada atrasada
resultados[resultados.loc[:,"Sem"] == 15]
resultados.dtypes
#transformar casa resul e visi resul em numeros

Sem           float64
Data           object
Em casa        object
Visitante      object
CasaResul      object
VisiResul      object
Vencedor       object
VCasaValor    float64
VVisValor     float64
count           int64
dtype: object

In [53]:
resultados['VCasaValor'] = np.where(resultados['Vencedor']=='C',1,np.where(resultados['Vencedor']=='E',0.5,0))
resultados['VVisValor'] = np.where(resultados['Vencedor']=='V',1,np.where(resultados['Vencedor']=='E',0.5,0))
resultados['count'] = 1


In [64]:
Casa = resultados.groupby(['Em casa'])[['count', 'VCasaValor','CasaResul','VisiResul']].sum().reset_index()
Casa = Casa.rename(columns={'Em casa':'time', 'CasaResul': 'GolsCasaC', 'VisiResul': 'GolsVisiC'})
Casa

,count,VCasaValor,GolsCasaC,GolsVisiC
Em casa,,,,
América (MG),19,6.0,0102221102200131003,3241022411021241332
Ath Paranaense,19,14.0,2231112232212131103,0122002030211121100
Atlético Mineiro,19,12.5,1221120112111023232,2101121200002101101
Bahia,19,10.5,1321211034111210104,2131202010120003111
Botafogo (RJ),19,13.0,2231222243311103310,1000000011021114410
Corinthians,19,11.0,2112103310411111111,1110111110401110152
Coritiba,19,6.0,0111003202220010110,3112011013403221012
Cruzeiro,19,8.0,1200001000101031211,0121110010101202211
Cuiabá,19,9.0,1101031230230100203,1241101102200112120


In [55]:
primeiro = resultados[resultados.loc[:, "Sem"] < 20]
segundo = resultados[resultados.loc[:, "Sem"] > 19]
print(primeiro.shape)
print(segundo.shape)

(190, 10)
(190, 10)
